## 웹서버로 연결하기

In [4]:
%%writefile scripts/simpleCoinbase.html
<!doctype>
<html>
<head>
<script src="https://cdn.jsdelivr.net/npm/web3@1.2.5/dist/web3.min.js"></script>
<!-- script src="https://cdn.jsdelivr.net/npm/web3@0.20.5/dist/web3.min.js"></script -->
<script type="text/javascript">
    var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

    function displayCoinbase() {
        
        document.getElementById('web3ver').innerText = 'web3 version 1.2.X';
        web3.eth.getCoinbase(function(err, coinbase){
            document.getElementById('web3coinbase').innerText = 'coinbase: ' + coinbase;
        })
    }
</script>
</head>
<body>
    <h1>Hello</h1>
    <button type="button" onClick="displayCoinbase();">Display Coinbase</button>
    <div></div>
    <div id="web3ver"></div>
    <div id="web3coinbase"></div>
</body>
</html>

Overwriting scripts/simpleCoinbase.html


### 파이썬 서버 실행
python -m http.server 8045  
localhost:8045/scripts

### sayHello 배포 및 사용

In [15]:
!node src/HelloDeploy.js

accounts: 0xcAda86CB06708c212978dFf4B8B2bDc0871e29B8
null 0x7d97eafa111fafa5f5d045d05172078fa7867da6e3789ff0e012eada5f04f32b
0x7aD8B1E0da91Fcf367f714D18CaD84379A106a9a


In [16]:
%%writefile scripts/hello.html
<!doctype>
<html>
<head>
<script src="https://cdn.jsdelivr.net/npm/web3@1.2.5/dist/web3.min.js"></script>
<script type="text/javascript">

    var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

    function displayHello() {

        document.getElementById('web3ver').innerText = 'web3 version 1.2.x';
        web3.eth.getCoinbase().then(function(coin) {
            document.getElementById('web3coinbase').innerText = 'coinbase: ' + coin;
            });
        
        var shelloContract = new web3.eth.Contract([{"constant":true,"inputs":[],"name":"sayHello","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"pure","type":"function"}],
                                      "0x7aD8B1E0da91Fcf367f714D18CaD84379A106a9a");
        
        shelloContract.methods.sayHello().call().then(function(str) {
            document.getElementById('sayhello').innerText = "The message from blockchain: " + str;
        });
        
    }
</script>
</head>
<body>
    <h1>Hello</h1>
    <button type="button" onClick="displayHello();">Display Hello</button>
    <div></div>
    <div id="sayhello"></div>
    <div id="web3ver"></div>
    <div id="web3coinbase"></div>
</body>
</html>

Overwriting scripts/hello.html


## greeter

### 1. 컨트랙 개발

In [2]:
%%writefile src/greeter.sol
pragma solidity ^0.6.0;

contract Greeter {
    string greeting;
    constructor() public {
        greeting='Hello';
    }
    function setGreeting(string memory _greeting) public {
        greeting = _greeting;
    }
    function greet() view public returns (string memory) {
        return greeting;
    }
}

Writing src/greeter.sol


### 2. 컴파일

In [3]:
!solc --abi src/greeter.sol


======= src/greeter.sol:Greeter =======
Contract JSON ABI
[{"inputs":[],"stateMutability":"nonpayable","type":"constructor"},{"inputs":[],"name":"greet","outputs":[{"internalType":"string","name":"","type":"string"}],"stateMutability":"view","type":"function"},{"inputs":[{"internalType":"string","name":"_greeting","type":"string"}],"name":"setGreeting","outputs":[],"stateMutability":"nonpayable","type":"function"}]


In [4]:
!solc --bin src/greeter.sol


======= src/greeter.sol:Greeter =======
Binary:
608060405234801561001057600080fd5b506040518060400160405280600581526020017f48656c6c6f0000000000000000000000000000000000000000000000000000008152506000908051906020019061005c929190610062565b50610107565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f106100a357805160ff19168380011785556100d1565b828001600101855582156100d1579182015b828111156100d05782518255916020019190600101906100b5565b5b5090506100de91906100e2565b5090565b61010491905b808211156101005760008160009055506001016100e8565b5090565b90565b610310806101166000396000f3fe608060405234801561001057600080fd5b50600436106100365760003560e01c8063a41368621461003b578063cfae3217146100f6575b600080fd5b6100f46004803603602081101561005157600080fd5b810190808035906020019064010000000081111561006e57600080fd5b82018360208201111561008057600080fd5b803590602001918460018302840111640100000000831117156100a257600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602

### 3. 배포 web3

In [5]:
%%writefile src/greeterDeploy.js

var Web3=require('web3');
var web3;

if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
}

var _abiArray=[{"constant":false,"inputs":[{"name":"_greeting","type":"string"}],"name":"setGreeting","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"greet","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var _bin="608060405234801561001057600080fd5b506040518060400160405280600581526020017f48656c6c6f0000000000000000000000000000000000000000000000000000008152506000908051906020019061005c929190610062565b50610107565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f106100a357805160ff19168380011785556100d1565b828001600101855582156100d1579182015b828111156100d05782518255916020019190600101906100b5565b5b5090506100de91906100e2565b5090565b61010491905b808211156101005760008160009055506001016100e8565b5090565b90565b610310806101166000396000f3fe608060405234801561001057600080fd5b50600436106100365760003560e01c8063a41368621461003b578063cfae3217146100f6575b600080fd5b6100f46004803603602081101561005157600080fd5b810190808035906020019064010000000081111561006e57600080fd5b82018360208201111561008057600080fd5b803590602001918460018302840111640100000000831117156100a257600080fd5b91908080601f016020809104026020016040519081016040528093929190818152602001838380828437600081840152601f19601f820116905080830192505050505050509192919290505050610179565b005b6100fe610193565b6040518080602001828103825283818151815260200191508051906020019080838360005b8381101561013e578082015181840152602081019050610123565b50505050905090810190601f16801561016b5780820380516001836020036101000a031916815260200191505b509250505060405180910390f35b806000908051906020019061018f929190610235565b5050565b606060008054600181600116156101000203166002900480601f01602080910402602001604051908101604052809291908181526020018280546001816001161561010002031660029004801561022b5780601f106102005761010080835404028352916020019161022b565b820191906000526020600020905b81548152906001019060200180831161020e57829003601f168201915b5050505050905090565b828054600181600116156101000203166002900490600052602060002090601f016020900481019282601f1061027657805160ff19168380011785556102a4565b828001600101855582156102a4579182015b828111156102a3578251825591602001919060010190610288565b5b5090506102b191906102b5565b5090565b6102d791905b808211156102d35760008160009055506001016102bb565b5090565b9056fea26469706673582212209b267c7ef3dcf37f2dffcbd963990eb71ece19f939b74800737072b119dfa18464736f6c63430006040033";
var _contract = new web3.eth.Contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
_contract
    .deploy({data:"0x"+_bin})
    .send({from: "0x2e9669337d8a86c70A984Ff5Bd307E05808909ED", gas: 364124, gasPrice: '1000000000'})
    .then(function(newContractInstance){
        console.log(newContractInstance.options.address) // instance with the new contract address
    });

Writing src/greeterDeploy.js


In [6]:
!node src/greeterDeploy.js

0x231834C584f54922a85AC69FdE9E2260eD8fDa59


### 4. 사용

In [14]:
%%writefile src/greeterUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
var _abiArray=[{"constant":false,"inputs":[{"name":"_greeting","type":"string"}],"name":"setGreeting","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"greet","outputs":[{"name":"","type":"string"}],"payable":false,"stateMutability":"view","type":"function"},{"inputs":[],"payable":false,"stateMutability":"nonpayable","type":"constructor"}];
var greeter = new web3.eth.Contract(_abiArray,"0x231834C584f54922a85AC69FdE9E2260eD8fDa59");
greeter.methods.greet().call().then(function(value) {console.log(value);});
greeter.methods.setGreeting("Hello SMU").send({from:"0x2f0d9a855f2D4f8C71461FD60EF88Ff07F3762df",gas:100000});
greeter.methods.greet().call().then(function(value) {console.log(value);});

Overwriting src/greeterUse.js


In [16]:
!node src/greeterUse.js

Hello SMU
Hello SMU


## Counter (계수기)

### 1. 컨트랙 개발

In [17]:
%%writefile src/Counter.sol
pragma solidity ^0.6.0;

contract Counter {
    uint256 counter = 0;
    function add() public {
        counter++;
    }
    function subtract() public {
        counter--;
    }
    function getCounter() public view returns (uint256) {
        return counter;
    }
}

Writing src/Counter.sol


### 2. 컴파일

In [18]:
!solc --abi --bin --gas src/Counter.sol


======= src/Counter.sol:Counter =======
Gas estimation:
construction:
   5099 + 42200 = 47299
external:
   add():	20959
   getCounter():	1035
   subtract():	20984
Binary:
60806040526000805534801561001457600080fd5b5060d3806100236000396000f3fe6080604052348015600f57600080fd5b5060043610603c5760003560e01c80634f2be91f1460415780636deebae31460495780638ada066e146051575b600080fd5b6047606d565b005b604f6080565b005b60576094565b6040518082815260200191505060405180910390f35b6000808154809291906001019190505550565b600080815480929190600190039190505550565b6000805490509056fea264697066735822122080d9d70ff6516aec61cfd97f52f17805751623584c3b022dc487fbfbf3efacde64736f6c63430006040033
Contract JSON ABI
[{"inputs":[],"name":"add","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"getCounter","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"subtract","outputs":[],"stateMutability":"nonpayable","type":"f

### 3. 배포

In [19]:
%%writefile src/counterDeploy.js

var Web3=require('web3');
var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
}

//solc 0.6.1
var _abiArray=[{"inputs":[],"name":"add","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"getCounter","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"subtract","outputs":[],"stateMutability":"nonpayable","type":"function"}];
var _bin="60806040526000805534801561001457600080fd5b5060d3806100236000396000f3fe6080604052348015600f57600080fd5b5060043610603c5760003560e01c80634f2be91f1460415780636deebae31460495780638ada066e146051575b600080fd5b6047606d565b005b604f6080565b005b60576094565b6040518082815260200191505060405180910390f35b6000808154809291906001019190505550565b600080815480929190600190039190505550565b6000805490509056fea2646970667358221220a19d7e0374295c3c6dd75807d6b2bb20a12deb6f736a4ad98c0065f0d9d4bf5764736f6c63430006010033";
var _contract = new web3.eth.Contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');
_contract
    .deploy({data:"0x"+_bin})
    .send({from: "0x2e9669337d8a86c70A984Ff5Bd307E05808909ED", gas: 364124, gasPrice: '1000000000'})
    .then(function(newContractInstance){
        console.log(newContractInstance.options.address) // instance with the new contract address
    });

Writing src/counterDeploy.js


In [20]:
!node src/counterDeploy.js

0xF6865ceE745d8Bd8EEbB202C5Ae40757c4e6ADBa


### 4. 사용

In [29]:
%%writefile src/counterUse.js
var Web3=require('web3');
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

var abi =[{"constant":false,"inputs":[],"name":"add","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":false,"inputs":[],"name":"subtract","outputs":[],"payable":false,"stateMutability":"nonpayable","type":"function"},{"constant":true,"inputs":[],"name":"getCounter","outputs":[{"name":"","type":"uint256"}],"payable":false,"stateMutability":"view","type":"function"}];
var addr = "0xF6865ceE745d8Bd8EEbB202C5Ae40757c4e6ADBa";
var counter = new web3.eth.Contract(abi,addr);

counter.methods.getCounter().call().then(function(str) {console.log(str);});
//counter.methods.subtract().send({from:"0x2e9669337d8a86c70A984Ff5Bd307E05808909ED",gas:100000});
//counter.methods.add().send({from:"0x2e9669337d8a86c70A984Ff5Bd307E05808909ED",gas:100000});
counter.methods.getCounter().call().then(function(str) {console.log(str);});

Overwriting src/counterUse.js


In [31]:
!node src/counterUse.js

3
3


## Timer

### 1. 컨트랙 개발

In [33]:
%%writefile src/Timer.sol
pragma solidity ^0.6.0;
contract Timer {
    uint256 startTime;
    function start() public {
        startTime=now;
    }
    function timePassed() public view returns(uint256) {
        return now-startTime;
    }
    function getNow() view public returns(uint) {
        return now;
    }
}

Writing src/Timer.sol


### 2. 컴파일
컴파일 결과를 js파일로 저장

In [161]:
!echo "exports._compiled=`solc --optimize --combined-json abi,bin,interface src/Timer.sol`" > src/Timer.js

In [74]:
!solc --optimize --combined-json abi,bin,interface src/Timer.sol

{"contracts":{"src/Timer.sol:Timer":{"abi":"[{\"inputs\":[],\"name\":\"getNow\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"start\",\"outputs\":[],\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"timePassed\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"}]","bin":"608060405234801561001057600080fd5b5060af8061001f6000396000f3fe6080604052348015600f57600080fd5b5060043610603c5760003560e01c8063b4454253146041578063bbe4fd50146059578063be9a655514605f575b600080fd5b60476067565b60408051918252519081900360200190f35b6047606f565b60656073565b005b600054420390565b4290565b4260005556fea26469706673582212208ffdfc5779131d9063dfa6180e084b2bed8f67a1151dd3b34494a2b8117b813964736f6c63430006040033"}},"version":"0.6.4+commit.1dca32f3.Windows.msvc"}


In [188]:
%%writefile src/Timer.js

exports._compiled={"contracts":{"src/Timer.sol:Timer":{"abi":"[{\"inputs\":[],\"name\":\"getNow\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"start\",\"outputs\":[],\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"timePassed\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"}]","bin":"608060405234801561001057600080fd5b5060af8061001f6000396000f3fe6080604052348015600f57600080fd5b5060043610603c5760003560e01c8063b4454253146041578063bbe4fd50146059578063be9a655514605f575b600080fd5b60476067565b60408051918252519081900360200190f35b6047606f565b60656073565b005b600054420390565b4290565b4260005556fea26469706673582212208ffdfc5779131d9063dfa6180e084b2bed8f67a1151dd3b34494a2b8117b813964736f6c63430006040033"}},"version":"0.6.4+commit.1dca32f3.Windows.msvc"}

Overwriting src/Timer.js


In [189]:
!type src\Timer.js


exports._compiled={"contracts":{"src/Timer.sol:Timer":{"abi":"[{\"inputs\":[],\"name\":\"getNow\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"start\",\"outputs\":[],\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"timePassed\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"}]","bin":"608060405234801561001057600080fd5b5060af8061001f6000396000f3fe6080604052348015600f57600080fd5b5060043610603c5760003560e01c8063b4454253146041578063bbe4fd50146059578063be9a655514605f575b600080fd5b60476067565b60408051918252519081900360200190f35b6047606f565b60656073565b005b600054420390565b4290565b4260005556fea26469706673582212208ffdfc5779131d9063dfa6180e084b2bed8f67a1151dd3b34494a2b8117b813964736f6c63430006040033"}},"version":"0.6.4+commit.1dca32f3.Windows.msvc"}


### require 테스트

In [77]:
%%writefile src/TimerImportTest.js
const my = require('./Timer');   //importing a javascript file
console.log("- compiled results: " +my);
console.log("- ABI: "+my._compiled.contracts['src/Timer.sol:Timer'].abi);
console.log("- Bytecode: "+my._compiled.contracts['src/Timer.sol:Timer'].bin);

Writing src/TimerImportTest.js


In [79]:
!node src\TimerImportTest.js

- compiled results: [object Object]
- ABI: [{"inputs":[],"name":"getNow","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"start","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"timePassed","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"}]
- Bytecode: 608060405234801561001057600080fd5b5060af8061001f6000396000f3fe6080604052348015600f57600080fd5b5060043610603c5760003560e01c8063b4454253146041578063bbe4fd50146059578063be9a655514605f575b600080fd5b60476067565b60408051918252519081900360200190f35b6047606f565b60656073565b005b600054420390565b4290565b4260005556fea26469706673582212208ffdfc5779131d9063dfa6180e084b2bed8f67a1151dd3b34494a2b8117b813964736f6c63430006040033


### 3. 배포

geth로 배포

!echo "var _compiled=`solc --optimize --combined-json abi,bin,interface src/Timer.sol`" > src/TimerGeth.js

In [86]:
!type src\TimerGeth.js

"var _compiled=`solc --optimize --combined-json abi,bin,interface src/Timer.sol`" 


In [87]:
!solc --optimize --combined-json abi,bin,interface src/Timer.sol

{"contracts":{"src/Timer.sol:Timer":{"abi":"[{\"inputs\":[],\"name\":\"getNow\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"start\",\"outputs\":[],\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"timePassed\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"}]","bin":"608060405234801561001057600080fd5b5060af8061001f6000396000f3fe6080604052348015600f57600080fd5b5060043610603c5760003560e01c8063b4454253146041578063bbe4fd50146059578063be9a655514605f575b600080fd5b60476067565b60408051918252519081900360200190f35b6047606f565b60656073565b005b600054420390565b4290565b4260005556fea26469706673582212208ffdfc5779131d9063dfa6180e084b2bed8f67a1151dd3b34494a2b8117b813964736f6c63430006040033"}},"version":"0.6.4+commit.1dca32f3.Windows.msvc"}


In [186]:
%%writefile src/TimerGeth.js
var _compiled={"contracts":{"src/Timer.sol:Timer":{"abi":"[{\"inputs\":[],\"name\":\"getNow\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"start\",\"outputs\":[],\"stateMutability\":\"nonpayable\",\"type\":\"function\"},{\"inputs\":[],\"name\":\"timePassed\",\"outputs\":[{\"internalType\":\"uint256\",\"name\":\"\",\"type\":\"uint256\"}],\"stateMutability\":\"view\",\"type\":\"function\"}]","bin":"608060405234801561001057600080fd5b5060af8061001f6000396000f3fe6080604052348015600f57600080fd5b5060043610603c5760003560e01c8063b4454253146041578063bbe4fd50146059578063be9a655514605f575b600080fd5b60476067565b60408051918252519081900360200190f35b6047606f565b60656073565b005b600054420390565b4290565b4260005556fea26469706673582212208ffdfc5779131d9063dfa6180e084b2bed8f67a1151dd3b34494a2b8117b813964736f6c63430006040033"}},"version":"0.6.4+commit.1dca32f3.Windows.msvc"}

Overwriting src/TimerGeth.js


In [112]:
%%writefile src/TimerDeployGeth.js
var primary = eth.accounts[1];
console.log("primary ac: ",primary);
console.log("balance: ",eth.getBalance(primary));
loadScript('src/TimerGeth.js')

contractName=Object.keys(_compiled.contracts)

_abi=JSON.parse(_compiled.contracts[contractName[0]].abi)
_code=_compiled.contracts[contractName[0]].bin
_class=eth.contract(_abi);

console.log('bin code: ', _code)


Overwriting src/TimerDeployGeth.js


In [111]:
!geth --exec "loadScript('src/TimerDeployGeth.js')" attach http://localhost:8445

primary ac:  0x9a47178d41d82a945199ea26f25318b44b3c1071
balance:  25000087786900010000
bin code:  608060405234801561001057600080fd5b5060af8061001f6000396000f3fe6080604052348015600f57600080fd5b5060043610603c5760003560e01c8063b4454253146041578063bbe4fd50146059578063be9a655514605f575b600080fd5b60476067565b60408051918252519081900360200190f35b6047606f565b60656073565b005b600054420390565b4290565b4260005556fea26469706673582212208ffdfc5779131d9063dfa6180e084b2bed8f67a1151dd3b34494a2b8117b813964736f6c63430006040033
GoError: Error while compiling or running script: Error: invalid address at web3.js:3930:11(47)
	at native
	at <eval>:1:12(1)



In [99]:

!geth --exec "eth.getTransactionReceipt('0x973734a68171e19143e2d5b2040378c9ea10e8aa77e674880d928b6d013c6f47')" attach http://localhost:8345

null


### node로 배포

In [121]:
%%writefile src/TimerDeployAbiBinFromFile.js
var Web3=require('web3');
var my = require('./Timer');  //cotaining abi,bin

var web3;
if (typeof web3 !== 'undefined') {
    web3 = new Web3(web3.currentProvider);
} else {
    web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));
}


//abi from Timer.js
var _abiArray=JSON.parse(my._compiled.contracts['src/Timer.sol:Timer'].abi);   //JSON parsing needed!!
console.log("- ABI: "+my._compiled.contracts['src/Timer.sol:Timer'].abi);

//bin from Timer.js
var _bin=my._compiled.contracts['src/Timer.sol:Timer'].bin;
console.log("- Bytecode: "+my._compiled.contracts['src/Timer.sol:Timer'].bin);

var _contract = new web3.eth.Contract(_abiArray);
//unlock the account with a password provided
//web3.personal.unlockAccount(web3.eth.accounts[0],'password');

_contract
    .deploy({data:"0x"+_bin})
    .send({from: "0x2e9669337d8a86c70A984Ff5Bd307E05808909ED", gas: 364124, gasPrice: '1000000000'})
    .then(function(newContractInstance){
        console.log("- Contract Address: "+newContractInstance.options.address) // instance with the new contract address
    });

Overwriting src/TimerDeployAbiBinFromFile.js


In [122]:
!node src/TimerDeployAbiBinFromFile.js

- ABI: [{"inputs":[],"name":"getNow","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"},{"inputs":[],"name":"start","outputs":[],"stateMutability":"nonpayable","type":"function"},{"inputs":[],"name":"timePassed","outputs":[{"internalType":"uint256","name":"","type":"uint256"}],"stateMutability":"view","type":"function"}]
- Bytecode: 608060405234801561001057600080fd5b5060af8061001f6000396000f3fe6080604052348015600f57600080fd5b5060043610603c5760003560e01c8063b4454253146041578063bbe4fd50146059578063be9a655514605f575b600080fd5b60476067565b60408051918252519081900360200190f35b6047606f565b60656073565b005b600054420390565b4290565b4260005556fea26469706673582212208ffdfc5779131d9063dfa6180e084b2bed8f67a1151dd3b34494a2b8117b813964736f6c63430006040033
- Contract Address: 0xCFfBbe3cE7F7B5ADC77F4bc15b47b98A47BE9bd3


## 4. 사용

In [199]:
%%writefile src/timerUse.js
var Web3=require('web3');
var my = require('./Timer');  //cotaining abi,bin
var web3 = new Web3(new Web3.providers.HttpProvider("http://localhost:8345"));

//abi from Timer.js
var _abiArray=JSON.parse(my._compiled.contracts['src/Timer.sol:Timer'].abi);   //JSON parsing needed!!
//console.log("- ABI: "+my._compiled.contracts['src/Timer.sol:Timer'].abi);

var timer = new web3.eth.Contract(_abiArray,"0xCFfBbe3cE7F7B5ADC77F4bc15b47b98A47BE9bd3");

//timer.methods.start().send({from:"0x2e9669337d8a86c70A984Ff5Bd307E05808909ED",gas:100000});
timer.methods.getNow().call().then(function(value) {console.log("now: " + value);});

timer.methods.timePassed().call().then(function(value) {console.log("passed: " + value);});

Overwriting src/timerUse.js


In [203]:
!node src/timerUse.js

now: 1619349883
passed: 0
